<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Plaid - Get accounts
[![download](https://img.shields.io/static/v1?label=Naas&labelColor=000000&message=Download&color=success&style=for-the-badge&logo=image%2Fsvg%2Bxml%3Bbase64%2CPD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz4KPHN2ZyB3aWR0aD0iMTAyNHB4IiBoZWlnaHQ9IjEwMjRweCIgdmlld0JveD0iMCAwIDEwMjQgMTAyNCIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIj4KIDwhLS0gR2VuZXJhdGVkIGJ5IFBpeGVsbWF0b3IgUHJvIDIuMC41IC0tPgogPGRlZnM%2BCiAgPHRleHQgaWQ9InN0cmluZyIgdHJhbnNmb3JtPSJtYXRyaXgoMS4wIDAuMCAwLjAgMS4wIDIyOC4wIDU0LjUpIiBmb250LWZhbWlseT0iQ29tZm9ydGFhLVJlZ3VsYXIsIENvbWZvcnRhYSIgZm9udC1zaXplPSI4MDAiIHRleHQtZGVjb3JhdGlvbj0ibm9uZSIgZmlsbD0iI2ZmZmZmZiIgeD0iMS4xOTk5OTk5OTk5OTk5ODg2IiB5PSI3MDUuMCI%2BbjwvdGV4dD4KIDwvZGVmcz4KIDx1c2UgaWQ9Im4iIHhsaW5rOmhyZWY9IiNzdHJpbmciLz4KPC9zdmc%2BCg%3D%3D)](https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Plaid/Plaid_Get_accounts.ipynb)

In [ ]:
pip install plaid-python

### Create account here :
https://plaid.com/
    
    

In [ ]:
import os
import plaid

## Config your variables

In [ ]:
PLAID_CLIENT_ID = "*************" 
PLAID_SECRET = "*************" 
PLAID_ENV = 'sandbox'

PLAID_PRODUCTS = ['transactions']
PLAID_COUNTRY_CODES = ['FR']
start_transaction = "2020-09-01"
end_transaction = "2020-10-01"

### Connect to plaid

In [ ]:
client = plaid.Client(client_id=PLAID_CLIENT_ID,
                      secret=PLAID_SECRET,
                      environment=PLAID_ENV)

In [ ]:
def create_link_token():
    response = client.LinkToken.create(
      {
        'user': {
          # This should correspond to a unique id for the current user.
          'client_user_id': 'user-id',
        },
        'client_name': "Plaid Quickstart",
        'products': PLAID_PRODUCTS,
        'country_codes': PLAID_COUNTRY_CODES,
        'language': "en",
        'redirect_uri': None,
      }
    )
    return response

In [ ]:
token = create_link_token()
token

### Use Naas callback to get the plaid OAuth token

In [ ]:
import naas
cb_url = naas.callback.add()

### Select Bank connection

In [ ]:
import IPython.core.display
import uuid

uid = uuid.uuid4().hex
iframe = """
<head>
  <script src="https://cdn.plaid.com/link/v2/stable/link-initialize.js"></script>
</head>
<script>
const handler_{uid} = Plaid.create({
  token: '{GENERATED_LINK_TOKEN}',
  onSuccess: (public_token, metadata) => {
        const xhr = new XMLHttpRequest();
        xhr.open("POST", "{CALLBACK_URL}", true);
        xhr.setRequestHeader('Content-Type', 'application/json');
        xhr.send(JSON.stringify({
            public_token: public_token
        }));
  }
});
handler_{uid}.open();
</script>
"""
iframe = iframe.replace('{uid}', uid)
iframe = iframe.replace('{CALLBACK_URL}', cb_url.get('url'))
iframe = iframe.replace('{GENERATED_LINK_TOKEN}', token.get('link_token'))
IPython.core.display.display(IPython.core.display.HTML(iframe))

### Get back plaid token

In [ ]:
import json

cb_data = naas.callback.get(cb_url.get('uuid'))
cb_data = json.loads(cb_data)
public_token = cb_data.get("public_token")
public_token

### Exange token 

In [ ]:
exchange_response = client.Item.public_token.exchange(public_token)
access_token = exchange_response['access_token']
item_id = exchange_response['item_id']

# Show account list

In [ ]:
import pandas as pd

response = client.Accounts.balance.get(access_token)
accounts = response['accounts']
accounts_df = pd.DataFrame.from_records(accounts)
accounts_df

# Save as csv

In [ ]:
accounts_df.to_csv('accounts.csv')

# if you need more data check the api doc 
https://plaid.com/docs/